### Imports

In [1]:
# Remove unwanted warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# Data Management
import polars as pl
import pandas as pd
from pandas_datareader.data import DataReader
from ta import add_all_ta_features
import yfinance as yf


# Statistics 
from statsmodels.tsa.stattools import adfuller

# Unsupervised Machine Learning
from sklearn.decomposition import PCA # Principle Component Analysis

# Supervised Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt



### Initial Data Extraction

In [2]:
# Data Extraction
import yfinance as yf

start_date = '2017-01-01'
end_date = '2022-06-01'
symbol = '^VIX'
df_pd = yf.download(symbol, start=start_date, end=end_date)
df_pd.head()


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-01-03,14.07,14.07,12.85,12.85,12.85,0
2017-01-04,12.78,12.80,11.63,11.85,11.85,0
2017-01-05,11.96,12.09,11.40,11.67,11.67,0
2017-01-06,11.70,11.74,10.98,11.32,11.32,0
2017-01-09,11.71,12.08,11.46,11.56,11.56,0


In [3]:
# Convert to Polars DataFrame

# Reset the index to make the date a column
df_pd.reset_index(inplace=True)

df = pl.from_pandas(df_pd)
print(df.head)

<bound method DataFrame.head of shape: (1_362, 7)
┌─────────────────────┬───────────┬───────────┬───────────┬───────────┬───────────┬────────┐
│ Date                ┆ Open      ┆ High      ┆ Low       ┆ Close     ┆ Adj Close ┆ Volume │
│ ---                 ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---    │
│ datetime[ns]        ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ i64    │
╞═════════════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪════════╡
│ 2017-01-03 00:00:00 ┆ 14.07     ┆ 14.07     ┆ 12.85     ┆ 12.85     ┆ 12.85     ┆ 0      │
│ 2017-01-04 00:00:00 ┆ 12.78     ┆ 12.8      ┆ 11.63     ┆ 11.85     ┆ 11.85     ┆ 0      │
│ 2017-01-05 00:00:00 ┆ 11.96     ┆ 12.09     ┆ 11.4      ┆ 11.67     ┆ 11.67     ┆ 0      │
│ 2017-01-06 00:00:00 ┆ 11.7      ┆ 11.74     ┆ 10.98     ┆ 11.32     ┆ 11.32     ┆ 0      │
│ 2017-01-09 00:00:00 ┆ 11.71     ┆ 12.08     ┆ 11.46     ┆ 11.56     ┆ 11.56     ┆ 0      │
│ …                 

In [4]:
df.schema


OrderedDict([('Date', Datetime(time_unit='ns', time_zone=None)),
             ('Open', Float64),
             ('High', Float64),
             ('Low', Float64),
             ('Close', Float64),
             ('Adj Close', Float64),
             ('Volume', Int64)])

In [5]:
# Convert the 'Date' column to the desired timezone (New York)
df = df.with_columns(
    pl.col('Date').dt.replace_time_zone('America/New_York')
)

print(df.head())

shape: (5, 7)
┌────────────────────────────────┬───────┬───────┬───────┬───────┬───────────┬────────┐
│ Date                           ┆ Open  ┆ High  ┆ Low   ┆ Close ┆ Adj Close ┆ Volume │
│ ---                            ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---       ┆ ---    │
│ datetime[ns, America/New_York] ┆ f64   ┆ f64   ┆ f64   ┆ f64   ┆ f64       ┆ i64    │
╞════════════════════════════════╪═══════╪═══════╪═══════╪═══════╪═══════════╪════════╡
│ 2017-01-03 00:00:00 EST        ┆ 14.07 ┆ 14.07 ┆ 12.85 ┆ 12.85 ┆ 12.85     ┆ 0      │
│ 2017-01-04 00:00:00 EST        ┆ 12.78 ┆ 12.8  ┆ 11.63 ┆ 11.85 ┆ 11.85     ┆ 0      │
│ 2017-01-05 00:00:00 EST        ┆ 11.96 ┆ 12.09 ┆ 11.4  ┆ 11.67 ┆ 11.67     ┆ 0      │
│ 2017-01-06 00:00:00 EST        ┆ 11.7  ┆ 11.74 ┆ 10.98 ┆ 11.32 ┆ 11.32     ┆ 0      │
│ 2017-01-09 00:00:00 EST        ┆ 11.71 ┆ 12.08 ┆ 11.46 ┆ 11.56 ┆ 11.56     ┆ 0      │
└────────────────────────────────┴───────┴───────┴───────┴───────┴───────────┴────────┘


In [6]:
# Add TA
# Convert back to Pandas DataFrame
df_pd = df.to_pandas()

# Add technical analysis features
df_pd = add_all_ta_features(df_pd, open='Open', high='High', low='Low', close='Adj Close', volume='Volume', fillna=True)

# Convert back to Polars Dataframe, remember need to have the pyarrow module installed
df = pl.from_pandas(df_pd)

In [7]:
df

Date,Open,High,Low,Close,Adj Close,Volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,…,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
"datetime[ns, America/New_York]",f64,f64,f64,f64,f64,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2017-01-03 00:00:00 EST,14.07,14.07,12.85,12.85,12.85,0,-0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,1000.0,12.85,12.85,12.85,0.0,0.0,0.0,0.0,13.256667,14.476666,12.036668,18.405823,0.333333,0.0,0.0,12.85,14.07,13.46,9.494158,0.0,0.0,…,13.46,13.46,0.0,0.0,0.0,0.0,0.0,20.47047,22.23312,0.0,0.0,0.0,10.94,14.07,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.85,0.0,0.0,0.0
2017-01-04 00:00:00 EST,12.78,12.8,11.63,11.85,11.85,0,-0.0,0,0.0,-0.0,0.0,0.0,0.0,0.0,50.0,1000.0,12.35,13.35,11.35,16.194331,0.25,0.0,0.0,12.675,13.87,11.48,18.856011,0.154812,0.0,0.0,11.63,14.07,12.85,19.757081,0.090164,0.0,…,12.85,12.85,0.0,0.0,0.0,0.0,-66.666667,20.47047,22.23312,0.0,4.0,-4.0,10.94,14.07,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,9.016406,9.016406,4.508203,-90.983594,0.0,0.0,-0.624394,-0.124879,-0.499515,0.0,0.0,0.0,12.098375,-7.782101,-8.101594,-7.782101
2017-01-05 00:00:00 EST,11.96,12.09,11.4,11.67,11.67,0,-0.0,0,0.0,-0.0,0.0,0.0,0.0,0.0,50.0,1000.0,12.123334,13.161452,11.085215,17.125952,0.281656,0.0,0.0,12.356667,13.383333,11.33,16.61721,0.165584,0.0,0.0,11.4,14.07,12.735,22.023646,0.101124,0.0,…,12.735,12.735,0.0,0.0,0.0,0.0,-70.740755,20.47047,22.23312,0.0,8.0,-8.0,10.94,14.07,0.0,1.0,0.0,0.0,0.0,0.0,-100.0,15.654975,10.112376,6.376261,-89.887624,0.0,0.0,-1.226732,-0.345249,-0.881483,0.0,0.0,0.0,11.82578,-1.51899,-1.530645,-9.182881
2017-01-06 00:00:00 EST,11.7,11.74,10.98,11.32,11.32,0,-0.0,0,0.0,-0.0,0.0,0.0,0.0,0.0,50.0,1000.0,11.9225,13.059283,10.785717,19.069543,0.234998,0.0,0.0,12.104167,13.064167,11.144167,15.862307,0.09158,0.0,0.0,10.98,14.07,12.525,25.917384,0.110032,0.0,…,12.525,12.525,0.0,0.0,0.0,0.0,-87.635601,20.47047,22.23312,0.0,12.0,-12.0,10.94,14.0166,0.0,0.0,0.0,0.0,0.0,0.0,-100.0,21.336782,11.003241,10.044008,-88.996759,0.0,0.0,-1.916831,-0.659566,-1.257265,0.0,0.0,0.0,11.503803,-2.999146,-3.045041,-11.90662
2017-01-09 00:00:00 EST,11.71,12.08,11.46,11.56,11.56,0,-0.0,0,0.0,-0.0,0.0,0.0,0.0,0.0,50.0,1000.0,11.85,12.907318,10.792683,17.845024,0.362861,0.0,0.0,12.023333,12.915333,11.131333,14.837815,0.240284,0.0,0.0,10.98,14.07,12.525,26.07595,0.187703,0.0,…,12.525,12.525,0.0,0.0,0.0,0.0,-41.34695,20.47047,22.23312,0.0,12.0,-12.0,10.94,13.895136,0.0,0.0,15.780578,0.0,0.0,0.0,-99.449116,23.010784,18.770254,13.29529,-81.229746,0.0,0.0,-2.289756,-0.985604,-1.304152,0.0,0.0,0.0,11.534477,2.120148,2.097985,-10.03891
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022-05-24 00:00:00 EDT,29.43,31.07,29.040001,29.450001,29.450001,0,0.0,0,0.0,-0.0,0.0,0.0,0.0,0